In [1]:
#####Sparse Factorization Machines#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
import scipy.linalg
import scipy.stats as ss
import itertools
from numpy.random import *
from scipy import optimize
from scipy.stats import norm
from scipy import sparse

In [22]:
##多項分布の乱数を生成する関数
def rmnom(pr, n, k, no):
    z_id = np.argmax((np.cumsum(pr, axis=1) > np.random.rand(n).reshape(n, 1)), axis=1)
    Z = sparse.coo_matrix((np.repeat(1, n), (no, np.array(z_id))), shape=(n, k))   #スパース行列の設定
    return Z

In [13]:
####データの発生####
##データの設定
k = 10   #基底数
item = 3000   #アイテム数
tag = 200   #タグ数
pt = np.random.poisson(np.random.gamma(30.0, 1/0.25, item), item)   #アイテムあたりの購買数
pt[pt < 2] = 2
N = np.sum(pt)   #総レコード数
n = np.random.poisson(1.75, item); n[n==0] = 1   #アイテムごとのタグ数
vec_k = np.repeat(1, k)

In [19]:
##IDとインデックスの設定
#IDの設定
item_id = np.repeat(range(item), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(item)])))

#インデックスの設定
index = np.arange(N)
item_list = [i for i in range(item)]
for i in range(item):
    item_list[i] = index[item_id==i]

In [20]:
##階層モデルの説明変数を生成
#アイテムの説明変数を生成
k1 = 3; k2 = 4; k3 = 4
v1 = np.array(np.random.random(item*k1)).reshape((item, k1))
v2 = np.zeros((item, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.55, 1)
    v2[:, j] = np.random.binomial(1, prob, item)
v3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(3.0, k3), 1).reshape(k3), item)
v3 = np.delete(v3, np.argmin(np.sum(v3, axis=0)), axis=1)   #冗長な変数の削除
v = np.concatenate((np.repeat(1, item)[:, np.newaxis], v1, v2, v3), axis=1)
column_v = v.shape[1]

In [87]:
##タグを生成
#パラメータの設定
topic = 25
omega = np.random.dirichlet(np.repeat(0.5, tag), topic)
z = np.dot(np.array(rmnom(np.random.dirichlet(np.repeat(0.5, topic), item), item, topic, np.arange(item)).todense()), np.arange(topic))

#多項分布からタグを生成
max_n = np.max(n)
tag_list = [j for j in range(item)]
tag_id = np.zeros((N, max_n), dtype="int")
tag_data = np.zeros((N, tag), dtype="int")

for j in range(item):
    while True:
        x = np.random.multinomial(n[j], omega[z[j], ], 1).reshape(-1)
        if np.max(x)==1:
            tag_list[j] = (x * (np.arange(tag)+1))[x > 0]
            break
    tag_id[item_list[j], 0:n[j]] = np.repeat(tag_list[j], pt[j]).reshape(pt[j], n[j], order="F")
    tag_data[item_list[j], ] = np.repeat(x, pt[j]).reshape(pt[j], tag, order="F")

tag_id0 = tag_id
tag_id0[tag_id0==0] = tag + 1

In [119]:
#組み合わせを作成
index_combine = np.array(list(itertools.combinations(np.arange(max_n), 2)))
combine_n = np.repeat(0, np.max(index_combine[:, 0])+1)
combine_list = [j for j in range(np.max(index_combine[:, 0])+1)]
for j in range(np.max(index_combine[:, 0]+1)):
    combine_list[j] = index_combine[np.array(np.where(index_combine[:, 0]==j)[0], dtype="int"), 1]
    combine_n[j] = combine_list[j].shape[0]

In [128]:
#交互作用項のインデックスを作成



array([[0, 1],
       [0, 2],
       [0, 3],
       [0, 4],
       [0, 5],
       [0, 6],
       [1, 2],
       [1, 3],
       [1, 4],
       [1, 5],
       [1, 6],
       [2, 3],
       [2, 4],
       [2, 5],
       [2, 6],
       [3, 4],
       [3, 5],
       [3, 6],
       [4, 5],
       [4, 6],
       [5, 6]])